### Fake News Detector

In [130]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [131]:
data_fake=pd.read_csv('Fake.csv')
data_true=pd.read_csv('True.csv')

In [132]:

print(data_fake.shape)
data_fake.sample(5)

(23481, 4)


,title,text,subject,date
16700,BRAINIAC Gets Rejected After Trying To Buy BMW...,Does anyone else out there see a future BMW ca...,Government News,"Mar 20, 2016"
158,Watch GOP Senator Almost Knock Over American ...,A Capitol Hill press conference that was suppo...,News,"October 31, 2017"
22050,Boiler Room EP #120 – Scorched Earth Media: Fr...,Tune in to the Alternate Current Radio Network...,US_News,"August 4, 2017"
15828,THE LIST OF OBAMA’S HISTORIC FIRSTS AKA HOW CH...,Wow! What a list of accomplishments! The probl...,politics,"Apr 14, 2015"
16713,BREAKING…Internal Memo From OBAMA’S Corrupt EP...,Wow! Bernie and Hillary have been making hay o...,Government News,"Mar 15, 2016"


In [133]:
print(data_true.shape)
data_true.sample(5)

(21417, 4)


,title,text,subject,date
18289,Ecuador president replaces vice president jail...,QUITO (Reuters) - Ecuador s president on Wedne...,worldnews,"October 4, 2017"
4986,Kansas senator stands by comparison of Planned...,(Reuters) - A Kansas state senator on Monday d...,politicsNews,"March 13, 2017"
10700,"Ben Carson says Obama was 'raised white,' spar...",NEW YORK (Reuters) - Republican presidential c...,politicsNews,"February 23, 2016"
12168,Brazil's Temer says pension bill 2 to 3 dozen ...,BRASILIA (Reuters) - Brazil s government has s...,worldnews,"December 15, 2017"
16141,"Spain's prosecutor calls for rebellion, sediti...",MADRID (Reuters) - Spain s state prosecutor on...,worldnews,"October 30, 2017"


In [134]:
data_fake_unseen = data_fake.tail(10)
data_true_unseen = data_true.tail(10)

for i in range(23480,23470,-1):
    data_fake.drop([i],axis = 0, inplace = True)

for i in range(21416,21406,-1):
    data_true.drop([i],axis = 0, inplace = True)

print(data_true.shape)

(21407, 4)


In [135]:
data_fake.shape, data_true.shape

((23471, 4), (21407, 4))

In [136]:
data_fake['class']=0
data_true['class']=1

In [137]:
data_fake_unseen['class']=0
data_true_unseen['class']=1

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5104\1714131927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_unseen['class']=0
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5104\1714131927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_unseen['class']=1


In [138]:
data_fake_unseen.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [139]:
data_true_unseen.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [140]:
df=pd.concat([data_fake, data_true], axis = 0)
df.sample(5)

,title,text,subject,date,class
1378,Alt-Right ‘Enemies’ List LEAKED: Fascists Use...,Last night Buzzfeed reported that a massive do...,News,"May 22, 2017",0
13190,LEO DITCHES HILLARY: Shady Excuse Doesn’t Fly ...,"If you believe this excuse, you ll believe any...",politics,"Aug 23, 2016",0
6033,‘My African-American’: Watch Donald Trump’s R...,Donald Trump pointed out a black person in the...,News,"June 3, 2016",0
1674,Scott Baio Gets His A** Handed To Him By Erin...,Erin Moran s family just declared war on Scott...,News,"April 26, 2017",0
907,Trump Was Just Busted For Daydreaming At G20 ...,"The last time Donald Trump went overseas, it w...",News,"July 7, 2017",0


In [141]:
df.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [142]:
df=df[['text','class']]

In [143]:
df=df.reset_index()

In [144]:
df.drop(columns=['index'])

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
44873,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,1
44874,WASHINGTON (Reuters) - The United States sugge...,1
44875,WASHINGTON (Reuters) - The United States has d...,1
44876,ISLAMABAD (Reuters) - Outlining a new strategy...,1


In [145]:

df.isnull().sum() 

index    0
text     0
class    0
dtype: int64

In [146]:
df.duplicated().sum()

np.int64(0)

#### Randomly shuffling the dataframe 

In [147]:
df = df.sample(frac=1)

In [148]:
df.head()

,index,text,class
12593,12593,CLINTON S STATE DEPARTMENT WASTE AND MISMANAGE...,0
27630,4159,WASHINGTON (Reuters) - U.S. Secretary of State...,1
19597,19597,Starbucks SJWs call cops on customer for reque...,0
34526,11055,"DES MOINES, Iowa (Reuters) - Even in boycottin...",1
224,224,Donald Trump s administration is probably the ...,0


In [149]:
df.reset_index(inplace = True)
df=df.drop(columns=['index'])
df=df.drop(columns=['level_0'])

In [150]:
df.columns

Index(['text', 'class'], dtype='object')

In [151]:
df.shape

(44878, 2)

In [152]:
df.head()

,text,class
0,CLINTON S STATE DEPARTMENT WASTE AND MISMANAGE...,0
1,WASHINGTON (Reuters) - U.S. Secretary of State...,1
2,Starbucks SJWs call cops on customer for reque...,0
3,"DES MOINES, Iowa (Reuters) - Even in boycottin...",1
4,Donald Trump s administration is probably the ...,0


### Preprocessing Text

In [153]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<.*?>+',b'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text = re.sub('\w*\d\w*','',text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\w'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5104\42366436.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]','',text)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5104\42366436.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+','',text)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5104\42366436.py:8: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*','',text)


In [154]:
df['text']=df['text'].apply(clean_text)

In [156]:
df['text']

0        clinton s state department waste and mismanage...
1        washington  reuters    u s  secretary of state...
2        starbucks sjws call cops on customer for reque...
3        des moines  iowa  reuters    even in boycottin...
4        donald trump s administration is probably the ...
                               ...                        
44873    london  reuters    london s police force urged...
44874    if there s one thing this election succeeded i...
44875    albany  new york  reuters    a new york judge ...
44876    massachusetts is now a sanctuary state  the ma...
44877    vienna  reuters    the head of austria s far r...
Name: text, Length: 44878, dtype: object

#### Defining dependent and independent variable as x and y

In [155]:
x = data['text']
y = data['class']

NameError: name 'data' is not defined

## Training the model

#### Splitting the dataset into training set and testing set. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

### Extracting Features from the Text

#### Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer(ngram_range=(1,3))
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

0.9863636363636363

In [ ]:
print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5801
           1       0.99      0.99      0.99      5419

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.9962566844919786

In [ ]:
print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5801
           1       0.99      0.99      0.99      5419

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Gradient Boost Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state=0,n_estimators=50,)
GB.fit(xv_train, y_train)

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\\backend\\cython\\checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
pred_gb = GB.predict(xv_test)

In [ ]:
GB.score(xv_test, y_test)

0.9961675579322639

In [ ]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5801
           1       0.99      1.00      1.00      5419

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=0,n_jobs=-1,n_estimators=25)
RF.fit(xv_train, y_train)

RandomForestClassifier(n_estimators=25, n_jobs=-1, random_state=0)

In [ ]:
pred_rf = RF.predict(xv_test)

In [ ]:
RF.score(xv_test, y_test)

0.993048128342246

In [ ]:
print (classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5801
           1       0.99      0.99      0.99      5419

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Testing the Model

In [ ]:
def output_lable(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    
    return print("\n\nLR Predicition: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction:{}".format(output_lable(pred_LR[0]),
                                                                                                             output_lable(pred_DT[0]),
                                                                                                             output_lable(pred_GB[0]),
                                                                                                             output_lable(pred_RF[0])))

### Model Testing With Manual Entry

In [ ]:
news=str(input())
manual_testing(news)

#down below text of fake news
#21st Century Wire says This week, the historic international Iranian Nuclear Deal was punctuated by a two-way prisoner swap between Washington and Tehran, but it didn t end quite the way everyone expected. On the Iranian side, one of the U.S. citizens who was detained in Iran, Nosratollah Khosravi-Roodsari, has stayed in Iran, but on the U.S. side   all 7 of the Iranians held in U.S. prisons DID NOT show up to their flight to Geneva for the prisoner exchange   with at least 3 electing to stay in the U.S  TEHRAN SIDE: In Iran, 5 U.S. prisoners were released, with 4 of them making their way to Germany via Switzerland.Will Robinson Daily MailNone of the Iranians freed in the prisoner swap have returned home and could still be in the United States, it has been reported.The seven former inmates, who were released as part of a deal with the Islamic republic, did not show up to get a flight to Geneva, Switzerland, where the exchange was set to take place on Sunday.Three of the Iranians have decided to stay in the United States, ABC reported, with some moving in with their families. However it is not known where the other four are.Three of the Americans who had been detained in Iran   Washington Post journalist Jason Rezaian, former U.S. Marine Amir Hekmati and Christian pastor Saeed Abedini   left Tehran at around 7am the same day, but weren t met by their counterparts in Switzerland Continue this story at the Mail OnlineREAD MORE IRAN NEWS AT: 21st Century Wire Iran Files



LR Predicition: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction:Fake News
